# AI 에이전트 기술 부여

## 1. 소개

시멘틱 커널 SDK의 핵심 기능 중 하나는 개발자가 네이티브 코드를 LLM(대규모 언어 모델)의 기능과 결합할 수 있다는 것입니다. 네이티브 기능이 포함된 플러그 인을 사용하여 AI(인공 지능) 에이전트의 기능을 확장할 수 있습니다. 이러한 네이티브 함수는 복잡한 계산부터 개인화된 권장 사항까지 광범위한 작업을 수행할 수 있습니다.

개인화된 듣기 환경을 제공할 수 있는 스마트 음악 서비스를 만들려고 한다고 가정해 보겠습니다. 네이티브 함수를 사용하여 음악 라이브러리와 상호 작용하고, 사용자의 "최근 재생" 목록에 노래를 추가한 다음 의미 체계 프롬프트를 사용하여 노래와 콘서트를 권장할 수 있습니다.

이 모듈에서는 의미 체계 커널 SDK에서 네이티브 함수가 작동하는 방식을 알아봅니다. 네이티브 함수가 사용자 지정된 작업을 수행하여 AI 에이전트에 "기술"을 효과적으로 제공하는 방법을 이해할 수 있습니다.

## 2. 네이티브 함수 이해

네이티브 함수는 애플리케이션에서 시멘틱 커널 SDK에서 호출할 수 있는 네이티브 코드로 작성된 함수입니다. LLM(대규모 언어 모델) 자체에서 수행할 수 없는 작업을 수행하는 데 유용합니다. 네이티브 함수는 애플리케이션이 수행할 수 있는 기술과 같다고 생각할 수 있습니다.

이후 모듈에서는 의미 체계 커널을 사용하여 사용자가 만든 네이티브 함수를 자동으로 호출하고 논리를 LLM에 프롬프트와 결합하는 방법을 알아봅니다. 채팅 완료 서비스와 기능 기술을 결합하여 다양한 작업을 수행할 수 있는 AI 에이전트를 만들 수 있습니다. 하지만 지금은 네이티브 함수를 디자인하는 데 집중해 보겠습니다.

네이티브 함수는 커널에서 사용할 특정 형식과 권장되는 파일 구조를 갖습니다. 네이티브 함수는 정의에 `KernelFunction` 데코레이터를 사용해야 합니다. 또한 매개 변수에 `Description` Attribute를 사용합니다. 

```csharp
[KernelFunction, Description("Convert an amount of currency to USD")]
public static string ConvertCurrency(
  [Description("The currency")] string currency, 
  [Description("The amount")] double amount)
{
  // Code to convert currency
}
```

네이티브 함수를 플러그 인으로 커널로 가져올 수 있습니다. 네이티브 함수를 포함하는 클래스는 "플러그 인" 디렉터리에 배치되어야 합니다. 코드를 구성하려면 관련 함수를 동일한 파일에 배치해야 합니다. "플러그 인" 디렉터리 내에서 하위 디렉터리를 활용하여 코드를 추가로 구성할 수도 있습니다.

예를 들어 할 일 목록 애플리케이션이 있다고 가정해 보겠습니다. 사용자가 할 일 목록에 있는 항목을 완료하려고 합니다. 대규모 언어 모델(LLM)은 사용자의 할 일 목록에 직접 액세스할 수 없지만 네이티브 함수를 작성하여 목록에 액세스하고 항목을 완료로 표시할 수 있습니다.

In [1]:
// 시멘틱커널 패키지 추가
#r "nuget:Microsoft.SemanticKernel"
#r "nuget:Microsoft.SemanticKernel.Plugins.Core,*-*"

using Microsoft.SemanticKernel;

Installed Packages Microsoft.SemanticKernel, 1.18.2 Microsoft.SemanticKernel.Plugins.Core, 1.18.2-alpha

In [2]:
using Kernel = Microsoft.DotNet.Interactive.Kernel;

var yourDeploymentName = await Kernel.GetInputAsync("OpenAI 배포 이름을 입력해주세요.").ConfigureAwait(false);
var yourEndpoint =  await Kernel.GetInputAsync("OpenAI Endpoint를 입력해주세요.").ConfigureAwait(false);
var yourApiKey = await Kernel.GetPasswordAsync("OpenAI API Key를 입력해주세요.").ConfigureAwait(false);
var yourModelId = "gpt-35-turbo-16k";

In [8]:
#!import Plugins/TodoListPlugin.cs

// 실험용 기능 오류처리 안하도록 선언
#pragma warning disable SKEXP0050

using Microsoft.SemanticKernel.Plugins.Core;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion(
    yourDeploymentName,
    yourEndpoint,
    yourApiKey.GetClearTextPassword(),
    yourModelId);

var kernel = builder.Build();

kernel.ImportPluginFromType<TodoListPlugin>();

var result = await kernel.InvokeAsync<string>(
  "TodoListPlugin",
  "CompleteTask",
  new() {{ "task", "코드 예제 완성하기" }}
).ConfigureAwait(false);

Console.WriteLine(result);

할 일 '코드 예제 완성하기'를 완료 처리하였습니다.


## +. 음악 라이브러리 플러그 인 만들기

1. `Plugins` 폴더에 `MusicLibrary.cs`라는 새 파일을 만듭니다.

2. `data`라는 새 폴더를 만들고 `recentlyplayed.txt`라는 새 파일을 만든 후 규칙에 맞는 새 콘텐츠를 만듭니다.

```json
[
    {
        "title": "하늘바라기",
        "artist": "정은지",
        "genre": "k-pop"
    },
    {
        "title": "Amateur",
        "artist": "이승철",
        "genre": "k-pop"
    },
    {
        "title": "Adventure",
        "artist": "Alexander Nakarada",
        "genre": "new-age"
    },
    {
        "title": "Zoltraak",
        "artist": "Evan Call",
        "genre": "new-age"
    }
]
```

In [9]:
#!import Plugins/MusicLibrary.cs

// 실험용 기능 오류처리 안하도록 선언
#pragma warning disable SKEXP0050

using Microsoft.SemanticKernel.Plugins.Core;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion(
    yourDeploymentName,
    yourEndpoint,
    yourApiKey.GetClearTextPassword(),
    yourModelId);

var kernel = builder.Build();

kernel.ImportPluginFromType<MusicLibraryPlugin>();

var result = await kernel.InvokeAsync(
    "MusicLibraryPlugin",
    "AddToRecentlyPlayed",
    new() {
        ["artist"] = "Tiara",
        ["song"] = "Danse",
        ["genre"] = "French pop, electropop, pop"
    }
).ConfigureAwait(false);

Console.WriteLine(result);

Error: File does not exist: 'Plugins/MusicLibraryPlugin.cs'.